In [1]:
!pip install datasets
from datasets import load_dataset
trainingDataset = load_dataset("wmt16","de-en", split="train[:50000]")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.2 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [2]:
validationDataset = load_dataset("wmt16","de-en", split="validation")
testingDataset = load_dataset("wmt16","de-en", split="test")

In [3]:
validationDataset

Dataset({
    features: ['translation'],
    num_rows: 2169
})

In [4]:
validationDataset[0]

{'translation': {'de': 'Die Premierminister Indiens und Japans trafen sich in Tokio.',
  'en': 'India and Japan prime ministers meet in Tokyo'}}

In [6]:
sentences = []
for example in validationDataset["translation"]:
  sentences.append(example['en'])
sentences

['India and Japan prime ministers meet in Tokyo',
 "India's new prime minister, Narendra Modi, is meeting his Japanese counterpart, Shinzo Abe, in Tokyo to discuss economic and security ties, on his first major foreign visit since winning May's election.",
 'Mr Modi is on a five-day trip to Japan to strengthen economic ties with the third largest economy in the world.',
 'High on the agenda are plans for greater nuclear co-operation.',
 'India is also reportedly hoping for a deal on defence collaboration between the two nations.',
 'Karratha police arrest 20-year-old after high speed motorcycle chase',
 'A motorcycle has been seized after it was ridden at 125km/h in a 70km/h zone and through bushland to escape police in the Pilbara.',
 'Traffic police on patrol in Karratha this morning tried to pull over a blue motorcycle when they spotted it reaching 125km/h as it pulled out of a service station on Bathgate Road.',
 'Police say the rider then failed to stop and continued on to Burgess

In [9]:
import torch
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "google/t5-small"
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small").to(device)

batch_size = 16
class EnglishDataset(torch.utils.data.Dataset):
    def __init__(self, sentences, tokenizer, max_length):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        input_text = f"translate English to German: {sentence}"
        input_ids = self.tokenizer.encode(input_text, max_length=self.max_length, truncation=True, padding="max_length")
        return torch.tensor(input_ids)

english_dataset = EnglishDataset(sentences, tokenizer, max_length=128)
english_dataloader = DataLoader(english_dataset, batch_size=batch_size)

translated_sentences = []
for batch in english_dataloader:
    batch = batch.to(device)
    with torch.no_grad():
        translated_ids = model.generate(batch, max_length=128, num_beams=4, early_stopping=True)
    translated_sentences.extend(tokenizer.batch_decode(translated_ids, skip_special_tokens=True))

for english_sentence, translated_sentence in zip(sentences, translated_sentences):
    print("English:", english_sentence)
    print("German:", translated_sentence)
    print()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Streaming output truncated to the last 5000 lines.
German: Gestern, nach 180 km Radfahren, kamen die Sportlerinnen und Sportler wieder in Bad Vilbel zurück.

English: Tens of Turkish Policemen Arrested over 'Plotting' against Gov't
German: Zehn türkische Polizisten verhaftet wegen "Plotting" gegen den Gouverneur

English: A total of 33 police officers have been detained in Turkey on suspicions of 'plotting against the government', local media outlets say.
German: Insgesamt wurden in der Türkei 33 Polizeibeamte wegen Verdachts, die Regierung zu bekämpfen, inhaftiert, so die lokalen Medien.

English: Police officials have not immediately commented.
German: Polizeibeamte haben nicht sofort Stellung genommen.

English: Among the detainees were 14 high-ranking officers, according to Hurriyet Daily News.
German: Unter den Gefangenen waren 14 hochrangige Offiziere, so Hurriyet Daily News.

English: Some of them were involved in last December's corruption probes targeting government officials,

In [12]:
translatedSentences = []
for example in validationDataset["translation"]:
  translatedSentences.append(example['de'])
translatedSentences

['Die Premierminister Indiens und Japans trafen sich in Tokio.',
 'Indiens neuer Premierminister Narendra Modi trifft bei seinem ersten wichtigen Auslandsbesuch seit seinem Wahlsieg im Mai seinen japanischen Amtskollegen Shinzo Abe in Toko, um wirtschaftliche und sicherheitspolitische Beziehungen zu besprechen.',
 'Herr Modi befindet sich auf einer fünftägigen Reise nach Japan, um die wirtschaftlichen Beziehungen mit der drittgrößten Wirtschaftsnation der Welt zu festigen.',
 'Pläne für eine stärkere kerntechnische Zusammenarbeit stehen ganz oben auf der Tagesordnung.',
 'Berichten zufolge hofft Indien darüber hinaus auf einen Vertrag zur Verteidigungszusammenarbeit zwischen den beiden Nationen.',
 'Polizei von Karratha verhaftet 20-Jährigen nach schneller Motorradjagd',
 'Ein Motorrad wurde beschlagnahmt, nachdem der Fahrer es mit 125 km/h in einer 70 km/h-Zone und durch Buschland gefahren hatte, um der Polizei in Bilbara zu entkommen.',
 'Verkehrspolizisten in Karratha versuchten heu

In [24]:
#!pip install evaluate
from datasets import load_metric
from evaluate import load
metric = load("bleu")
bleu_score = metric.compute(predictions=translated_sentences, references=translatedSentences)
meteor = load('meteor')
meteor_score = meteor.compute(predictions=translated_sentences, references=translatedSentences)
bertscore = load("bertscore")
bertscore_score = bertscore.compute(predictions=translated_sentences, references=translatedSentences, lang = "en")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 883.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [25]:
print(bleu_score)
print(meteor_score)
print(bertscore_score)

{'bleu': 0.28210531208473444, 'precisions': [0.5944094820876604, 0.33733644859813083, 0.21676428606585618, 0.14571636514867956], 'brevity_penalty': 1.0, 'length_ratio': 1.0160189787618616, 'translation_length': 44969, 'reference_length': 44260}
{'meteor': 0.5518401805312027}
{'precision': [0.9825904965400696, 0.9256643056869507, 0.9556907415390015, 0.9131982326507568, 0.931688129901886, 0.8891658186912537, 0.9760649800300598, 0.900287389755249, 0.9374041557312012, 0.9659603834152222, 0.9057344198226929, 0.9562430381774902, 0.9412723183631897, 0.9004175066947937, 0.9174317121505737, 0.8865144848823547, 0.9716230034828186, 0.8619217276573181, 0.9087929725646973, 0.9044854044914246, 0.9132551550865173, 0.9362432360649109, 0.9032161831855774, 0.8619306087493896, 0.9015508890151978, 0.9040396213531494, 0.9334425926208496, 0.913665235042572, 0.9289020895957947, 0.9117105007171631, 0.9561638236045837, 0.8940197825431824, 0.9464868903160095, 0.9061379432678223, 0.9021632671356201, 0.9454046487